In [1]:
%load_ext autoreload
%autoreload 2
import sys,os
import torch
current_directory = os.getcwd()
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(current_directory))))))
from datasets.weather_bench import WeatherDataset

In [2]:
device = ("cuda" if torch.cuda.is_available() else "cpu" )
device = torch.device(device)

weather = WeatherDataset(0, device=device, normalize=True)
# dataset.shape:  torch.Size([7309, 100, 1450])
original = weather.load()

데이터셋 불러오는 중...
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, latitude: 28, longitude: 28,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float32 39.0 38.75 38.5 ... 32.5 32.25
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float32 124.2 124.5 ... 130.8 131.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, latitude, longitude) float32 ...
    10m_v_component_of_wind   (time, latitude, longitude) float32 ...
    2m_temperature            (time, latitude, longitude) float32 ...
    geopotential              (time, level, latitude, longitude) float32 ...
    mean_sea_level_pressure   (time, latitude, longitude) float32 ...
    sea_surface_temperature   (time, latitude, longitude) float32 ...
    ...                        ...
    total_cloud

Processing futures: 14it [00:13,  1.04it/s]
Processing futures: 2it [00:01,  1.53it/s]


15.61906 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, longitude: 14, latitude: 33,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float64 21.0 22.5 24.0 ... 67.5 69.0
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float64 121.5 123.0 ... 139.5 141.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, longitude, latitude) float32 ...
    10m_v_component_of_wind   (time, longitude, latitude) float32 ...
    2m_temperature            (time, longitude, latitude) float32 ...
    geopotential              (time, level, longitude, latitude) float32 ...
    mean_sea_level_pressure   (time, longitude, latitude) float32 ...
    sea_surface_temperature   (time, longitude, latitude) float32 ...
    ...                        ...
    total_cloud_cov

Processing futures: 14it [00:10,  1.33it/s]
Processing futures: 2it [00:00,  6.24it/s]


11.11322 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, longitude: 17, latitude: 12,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float64 2.903 8.71 ... 60.97 66.77
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float64 90.0 95.62 ... 174.4 180.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, longitude, latitude) float32 ...
    10m_v_component_of_wind   (time, longitude, latitude) float32 ...
    2m_temperature            (time, longitude, latitude) float32 ...
    geopotential              (time, level, longitude, latitude) float32 ...
    mean_sea_level_pressure   (time, longitude, latitude) float32 ...
    sea_surface_temperature   (time, longitude, latitude) float32 ...
    ...                        ...
    total_cloud_cover 

Processing futures: 14it [00:09,  1.41it/s]
Processing futures: 2it [00:00, 14.58it/s]


10.23150 sec
======= RESULT SHAPE =======
result_dataset.shape:  torch.Size([7305, 100, 1450])


In [34]:
%load_ext autoreload
%autoreload 2
from models.VariableAnalyzer.datasets.dataset import CustomDataset
dataset = CustomDataset(original, 4 * 7, device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
print(dataset[3])

tensor([[0.2695, 0.2715, 0.2703,  ..., 0.7818, 0.6788, 0.5188],
        [0.1810, 0.1773, 0.1726,  ..., 0.4673, 0.4184, 0.3465],
        [0.1628, 0.1569, 0.1506,  ..., 0.3565, 0.3355, 0.2872],
        ...,
        [0.3319, 0.3644, 0.3936,  ..., 0.2570, 0.4454, 0.6686],
        [0.2312, 0.4194, 0.4887,  ..., 0.2219, 0.3896, 0.6130],
        [0.3171, 0.0150, 0.0218,  ..., 0.1309, 0.2664, 0.4330]],
       device='cuda:0')


In [36]:
from torch.utils.data import DataLoader, Dataset

dataloader = DataLoader(dataset, 32, shuffle=True)

In [37]:
print(len(dataloader))

228


In [38]:
for i, batch in enumerate(dataloader):
    print(i, batch.shape)

0 torch.Size([32, 2900, 1450])
1 torch.Size([32, 2900, 1450])
2 torch.Size([32, 2900, 1450])
3 torch.Size([32, 2900, 1450])
4 torch.Size([32, 2900, 1450])
5 torch.Size([32, 2900, 1450])
6 torch.Size([32, 2900, 1450])
7 torch.Size([32, 2900, 1450])
8 torch.Size([32, 2900, 1450])
9 torch.Size([32, 2900, 1450])
10 torch.Size([32, 2900, 1450])
11 torch.Size([32, 2900, 1450])
12 torch.Size([32, 2900, 1450])
13 torch.Size([32, 2900, 1450])
14 torch.Size([32, 2900, 1450])
15 torch.Size([32, 2900, 1450])
16 torch.Size([32, 2900, 1450])
17 torch.Size([32, 2900, 1450])
18 torch.Size([32, 2900, 1450])
19 torch.Size([32, 2900, 1450])
20 torch.Size([32, 2900, 1450])
21 torch.Size([32, 2900, 1450])
22 torch.Size([32, 2900, 1450])
23 torch.Size([32, 2900, 1450])
24 torch.Size([32, 2900, 1450])
25 torch.Size([32, 2900, 1450])
26 torch.Size([32, 2900, 1450])
27 torch.Size([32, 2900, 1450])
28 torch.Size([32, 2900, 1450])
29 torch.Size([32, 2900, 1450])
30 torch.Size([32, 2900, 1450])
31 torch.Size([32,

In [39]:
from models.VariableAnalyzer.models.model import VariableAnalyzer

model = VariableAnalyzer(100, 4 * 7, 1450, 25, 3, 3)

for i, batch in enumerate(dataloader):
    print(model.get_tgt_mask(batch))
    if i == 0:
        break

/workspace/Haea/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


tensor([[[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         ...,
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., 27., 27., 27.]],

        [[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         ...,
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., 27., 27., 27.]],

        [[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         ...,
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., 27., 27., 27.]],

       

In [6]:
import torch

def create_custom_lower_triangular_matrix(size, interval):
    # 모든 요소가 0인 행렬 생성
    matrix = torch.zeros(size, size)
    
    # 대각선을 따라 interval 간격으로 1 채우기
    for i in range(0, size):
        for j in range(0, min(interval*(i+1), size)):
            matrix[i, j] = 1
    
    print(matrix)
    # Lower triangular matrix 반환
    return torch.tril(matrix)



size = 5
interval = 2
custom_matrix = create_custom_lower_triangular_matrix(size, interval)
print(custom_matrix)

tensor([[1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])


In [26]:
def get_time_seq(src: torch.Tensor):
        time_seq = []
        for i in range(4 * 7):
            seq = [i for _ in range(100)]
            time_seq.extend(seq)
        return torch.tensor([time_seq for _ in range(src.size(0))])


for i, batch in enumerate(dataloader):
    print()
    if i == 0:
        break

(slice(0, 3, None), 1)


TypeError: list indices must be integers or slices, not tuple